In [1]:
import numpy as np
from astropy import units as u
import pandas as pd

'''
    NAME:
         convert_ecl_rel_pos_to_geo_rel_ast
         
    PURPOSE:
         To determine the deltaLat and deltaLong of a moon from its primary KBO.
         
    CALLING SEQUENCE:
          deltaLong,deltaLat = convert_ecl_rel_pos_to_geo_rel_ast(ecl_rel_pos, obs_obj_rel_pos)
   
    INPUTS
          ecl_rel_pos - The J2000 ecliptic relative position of the KBO in Cartesian coordinates
          obs_obj_rel_pos - The J2000 ecliptic relative position of the Moon in Cartesian coordinates
   
    OUTPUTS:
          deltaLong - The difference in Longitude of the moon vs. it's primary KBO
          deltaLat - The difference in Latitude of the moon vs. it's primary KBO
'''
def convert_ecl_rel_pos_to_geo_rel_ast(ecl_rel_pos, obj_rel_pos, rel_moon):
    
    #Get the Cartesian positions of the Observer
    x1,y1,z1 = ecl_rel_pos[0],ecl_rel_pos[1],ecl_rel_pos[2]
    #Get the distance from the origin (Heliocenter) of the observer
    R1= np.sqrt(x1**2+y1**2+z1**2)
    

    #Get the Heliocentric Cartesian positions of the KBO
    x2,y2,z2 = obj_rel_pos[0],obj_rel_pos[1],obj_rel_pos[2]
    
    #Observer centric coordinates on KBO
    x2,y2,z2 = x2-x1,y2-y1,z2-z1
    
    #Get the distance from the origin (Now observer-center) of the KBO
    R2= np.sqrt(x2**2+y2**2+z2**2)
    
    x3,y3,z3 = rel_moon[0],rel_moon[1],rel_moon[2]
    moonX = x3+x2
    moonY = y3+y2
    moonZ = z3+z2
    R3 = np.sqrt((x3+x2)**2+(y3+y2)**2+(z3+z2)**2)
    
    
    #Now calculate the latitude and longitude from the coordinates given
    longitude1 = np.arcsin(z1/R1*u.degree)
    latitude1 = np.arccos(x1/R1/np.cos(longitude1*u.degree)*u.degree)
    
    #Calculate the latitude and longitude from the coordinates
    longitude2 = np.arcsin(z2/R2*u.degree)
    latitude2 = np.arccos(x2/R2/np.cos(longitude2*u.degree)*u.degree)
    
     #Calculate the latitude and longitude from the coordinates
    longitude3 = np.arcsin(moonZ/R3*u.degree)
    latitude3 = np.arccos(moonX/R3/np.cos(longitude3*u.degree)*u.degree)
    
    #Calculate the deltaLat and deltaLong
    deltaLat = latitude3-latitude2
    deltaLong = (longitude3-longitude2)*np.cos(latitude2)    

    return deltaLong, deltaLat


In [19]:
from astroquery.jplhorizons import Horizons
from astropy.time import Time

df = pd.read_csv('NewMakeMake_LatLon.csv')   
x1 = df['X1']
x2 = df['X2']
y1 = df['Y1']
y2 = df['Y2']
z1 = df['Z1']
z2 = df['Z2']

dates = df['Primary-Centric Time']
dateList = []
for i in dates:
    jd = Time(i,format='jd')
    dateList.append(jd)


GAIA = Horizons(id='258',location='299',epochs=dateList)
Earth = Horizons(id='299',location='299',epochs=dateList)
MakeMake = Horizons(id='MakeMake',location='299',epochs=dateList)

xGAIA = GAIA.vectors()['x']
yGAIA = GAIA.vectors()['y']
zGAIA = GAIA.vectors()['z']
xEarth = Earth.vectors()['x']
yEarth = Earth.vectors()['y']
zEarth = Earth.vectors()['z']
xMakeMake = MakeMake.vectors()['x']
yMakeMake = MakeMake.vectors()['y']
zMakeMake = MakeMake.vectors()['z']

print('GAIA: \n',xGAIA, '\n', yGAIA, '\n',zGAIA, '\n')
print('Earth: \n',xEarth, '\n', yEarth, '\n',zEarth, '\n')
print('MakeMake: \n',xMakeMake, '\n', yMakeMake, '\n',zMakeMake, '\n')



EclLatL2 = GAIA.ephemerides()['EclLat','ObsEclLat']
EclLonL2 = GAIA.ephemerides()['EclLon','ObsEclLon']
EarthLat = Earth.ephemerides()['EclLat','ObsEclLat']
EarthLon = Earth.ephemerides()['EclLon','ObsEclLon']
#print(EclLatL2,'\n',EclLonL2,'\n', EarthLat,'\n', EarthLon)

convert_ecl_rel_pos_to_geo_rel_ast([x1,y1,z1],[x2,y2,z2],[5,5,5])

GAIA: 
         x        
        AU       
-----------------
1.467532934354569
1.467119921443311
1.466266870363826
1.466060356501177
1.465854023056826 
         y        
        AU       
-----------------
2.821763184500026
2.822146118837983
2.822939026410739
2.823131334746641
 2.82332359394539 
          z         
         AU        
-------------------
-0.4241169656169576
-0.4241639612861788
-0.4242619009725043
-0.4242857869099143
  -0.42430971877003 

Earth: 
         x        
        AU       
-----------------
2.242718691529757
2.242694084124862
2.242643578543454
2.242631353059558
2.242619117017767 
         y         
        AU        
------------------
-1.342577266631915
-1.342147867851709
-1.341258829016537
-1.341043223160975
-1.340827679578346 
          z         
         AU        
-------------------
0.06117033900683319
0.06117357324988975
0.06117940960712887
0.06118064442866297
0.06118180770028402 

MakeMake: 
         x         
        AU        
-----------------

ConnectionError: HTTPSConnectionPool(host='ssd.jpl.nasa.gov', port=443): Read timed out.